In [72]:
from riotwatcher import LolWatcher, ApiError

lol_watcher = LolWatcher('RGAPI-a13f9fb8-4bcc-411f-93a1-65583711f27e')

my_region = 'eun1'

me = lol_watcher.summoner.by_name(my_region, 'xzxxsx')
print(me)

# all objects are returned (by default) as a dict
# lets see if I got diamond yet (I probably didn't)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

# Lets get some champions
# static_champ_list = lol_watcher.static_data.champions(my_region)
# print(static_champ_list)

# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

try:
    response = lol_watcher.summoner.by_name(my_region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

{'id': '5kbw3hTPLFJKW-85vpzyte9QspyLT0oswOfE2406yR_oHGw', 'accountId': 'beTVQanF42n6M_LZgahTXDq-dElU0Dt4KF05EkX0rhP9www', 'puuid': 'vn_y7TdJocGWRknEvYOjPFJwIl0iSw1vNOi4xPaU10UvsxDMc01KSX7g0edVKv_0D9Zihm0V1Bv4PQ', 'name': 'xzxxsx', 'profileIconId': 1213, 'revisionDate': 1595785601000, 'summonerLevel': 100}
[{'leagueId': 'e1f24cd8-e022-43ea-afa8-c08153613ace', 'queueType': 'RANKED_FLEX_SR', 'tier': 'BRONZE', 'rank': 'I', 'summonerId': '5kbw3hTPLFJKW-85vpzyte9QspyLT0oswOfE2406yR_oHGw', 'summonerName': 'xzxxsx', 'leaguePoints': 74, 'wins': 17, 'losses': 19, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]
Summoner with that ridiculous name not found.


In [90]:
import pandas as pd

my_matches = lol_watcher.match.matchlist_by_account(my_region, me['accountId'])
# fetch last match detail
last_match = my_matches['matches'][9]
match_detail = lol_watcher.match.by_id(my_region, last_match['gameId'])
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['gameID'] = last_match['gameId']
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,2540956801,11,4,11,True,7,11,15,344425,18412,18,45,1400,3031
1,2540956801,36,4,12,True,7,8,21,210984,17109,18,189,3143,3009
2,2540956801,202,4,7,True,11,11,17,205405,17623,18,184,3095,3031
3,2540956801,45,4,12,True,20,9,12,419952,24877,18,265,3089,3285
4,2540956801,98,4,14,True,6,9,19,122287,15758,18,76,3068,3860
5,2540956801,222,4,3,False,6,13,12,204611,14803,18,154,3140,3031
6,2540956801,517,4,14,False,15,12,9,204433,17834,18,153,3089,3157
7,2540956801,6,12,4,False,18,8,12,362725,23507,18,271,3153,3071
8,2540956801,54,4,14,False,2,9,14,76827,12857,18,44,3853,3285
9,2540956801,64,11,4,False,7,9,12,283389,17525,18,88,1400,3053


In [91]:
# check league's latest version
# latest = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
# Lets get some champions static information
static_champ_list = lol_watcher.data_dragon.champions("10.15.1", False, 'en_US')
# champ static list data to dict for looking up
champ_dict = {}
for key in static_champ_list['data']:
    row = static_champ_list['data'][key]
    champ_dict[row['key']] = row['id']
for row in participants:
    print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
    row['championName'] = champ_dict[str(row['champion'])]
# print dataframe
df = pd.DataFrame(participants)
df

11 MasterYi
36 DrMundo
202 Jhin
45 Veigar
98 Shen
222 Jinx
517 Sylas
6 Urgot
54 Malphite
64 LeeSin


,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,championName
0,2540956801,11,4,11,True,7,11,15,344425,18412,18,45,1400,3031,MasterYi
1,2540956801,36,4,12,True,7,8,21,210984,17109,18,189,3143,3009,DrMundo
2,2540956801,202,4,7,True,11,11,17,205405,17623,18,184,3095,3031,Jhin
3,2540956801,45,4,12,True,20,9,12,419952,24877,18,265,3089,3285,Veigar
4,2540956801,98,4,14,True,6,9,19,122287,15758,18,76,3068,3860,Shen
5,2540956801,222,4,3,False,6,13,12,204611,14803,18,154,3140,3031,Jinx
6,2540956801,517,4,14,False,15,12,9,204433,17834,18,153,3089,3157,Sylas
7,2540956801,6,12,4,False,18,8,12,362725,23507,18,271,3153,3071,Urgot
8,2540956801,54,4,14,False,2,9,14,76827,12857,18,44,3853,3285,Malphite
9,2540956801,64,11,4,False,7,9,12,283389,17525,18,88,1400,3053,LeeSin


In [44]:
df['championName']

,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,championName
0,18,4,7,True,8,6,3,165814,12727,16,181,3095,3156,Tristana
1,53,4,14,True,3,5,11,15876,7780,15,10,3855,3109,Blitzcrank
2,50,14,4,True,8,6,7,154657,14149,17,196,1056,3157,Swain
3,141,4,11,True,10,5,0,231189,13710,17,52,3142,3812,Kayn
4,17,4,12,True,1,7,4,188174,11688,17,222,3151,3115,Teemo
5,202,4,21,False,3,6,9,150545,11373,15,208,3094,3009,Jhin
6,67,4,14,False,5,4,4,116363,11686,17,191,1054,3153,Vayne
7,29,4,11,False,9,4,7,160412,13266,17,37,1416,3153,Twitch
8,54,3,4,False,3,9,9,33173,7957,13,46,3860,3285,Malphite
9,105,4,14,False,9,7,5,116276,12433,15,180,1052,3152,Fizz


In [71]:
razvy_games = set(pd.DataFrame(my_matches['matches']).gameId.unique())
razvy_games

{2429348100,
 2430072782,
 2430098147,
 2430610856,
 2430614570,
 2430651237,
 2430665953,
 2430776574,
 2430782852,
 2430789123,
 2430888205,
 2431123096,
 2431147704,
 2431409827,
 2431423075,
 2431836969,
 2432151161,
 2432198516,
 2432210163,
 2433091295,
 2433285048,
 2433289736,
 2433321760,
 2433478265,
 2433480493,
 2433502878,
 2433505305,
 2433891278,
 2433905508,
 2433983055,
 2434007343,
 2434127976,
 2434384510,
 2434602542,
 2434689359,
 2434943520,
 2435564905,
 2435596111,
 2435610849,
 2435670955,
 2435729307,
 2436694029,
 2436896351,
 2437022677,
 2438279771,
 2438341682,
 2438447525,
 2438527993,
 2439490648,
 2439577171,
 2439656246,
 2440693789,
 2443422104,
 2443435779,
 2443604776,
 2453192004,
 2453217361,
 2455206303,
 2455361791,
 2493490072,
 2493551724,
 2497467000,
 2497468038,
 2497512410,
 2504011441,
 2533773192,
 2533810830,
 2534482754,
 2535196366,
 2535288474,
 2535943059,
 2535998106,
 2536012888,
 2536014803,
 2536029378,
 2536652026,
 2536657188,

In [74]:
rary_games = set(pd.DataFrame(my_matches['matches']).gameId.unique())

In [93]:
my_matches['matches

[{'platformId': 'EUN1',
  'gameId': 2542786996,
  'champion': 29,
  'queue': 440,
  'season': 13,
  'timestamp': 1595783479045,
  'role': 'NONE',
  'lane': 'JUNGLE'},
 {'platformId': 'EUN1',
  'gameId': 2542119244,
  'champion': 202,
  'queue': 440,
  'season': 13,
  'timestamp': 1595714733231,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'EUN1',
  'gameId': 2542053957,
  'champion': 202,
  'queue': 440,
  'season': 13,
  'timestamp': 1595711825687,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'EUN1',
  'gameId': 2541204029,
  'champion': 202,
  'queue': 440,
  'season': 13,
  'timestamp': 1595630416177,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'EUN1',
  'gameId': 2541240350,
  'champion': 141,
  'queue': 440,
  'season': 13,
  'timestamp': 1595628248900,
  'role': 'NONE',
  'lane': 'JUNGLE'},
 {'platformId': 'EUN1',
  'gameId': 2541175731,
  'champion': 141,
  'queue': 440,
  'season': 13,
  'timestamp': 1595625736031,
  'role': 'NONE',